# 교차 검증과 그리드 서치
## 검증 세트

- 앞에서는 훈련 세트(80%) : 테스트 세트(20%) 비율로 나눴음.

- 검증세트는 훈련세트에서 추가로 20% 정도를 떼어냄.

- 검증세트는 테스트 세트를 사용하지 않고 모델이 과대적합인지 과소적합인지 파악하기 위해 사용.

- 훈련 세트에서 모델을 훈련 => 검증 세트로 모델을 평가

- 이런 식으로 테스트하고 싶은 매개변수를 바꿔가며 가장 좋은 모델을 고름

- 그 다음 이 매개변수를 사용해 훈련 세트와 검증 세트를 합쳐 전체 훈련 데이터에서 모델을 다시 훈련

- 마지막에 테스트 세트에서 최종 점수를 평가

In [2]:
# 데이터 불러오기
import pandas as pd
wine = pd.read_csv('./db/wine_csv_data.csv')

In [3]:
# class 열을 타깃으로 사용하고 나머지 열은 특성 배열에 저장
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [4]:
# 훈련 세트와 테스트 세트 나누기
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [5]:
# train_input & train_target 을 다시 훈련 세트와 검증세트로 나누기.
# train_input 의 20%를 val_input 으로 만들기
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [6]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [7]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


- 훈련 세트에 과대적합 되어있음 => 매개변수를 바꿔서 더 좋은 모델을 찾아야 함.

## 교차 검증(cross validation)

- 교차 검증은 검증 세트를 떼어 내어 평가하는 과정을 여러번 반복

- 그 다음 이 점수를 평균하여 최종 검증 점수를 얻음

- 사이킷런의 cross_validate() 라는 교차 검증 함수를 사용

In [8]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00539112, 0.00524807, 0.00541306, 0.00601912, 0.00519204]), 'score_time': array([0.00046897, 0.00039291, 0.00041389, 0.00047207, 0.00044918]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [9]:
# 교차 점증의 최종 점수는 test_score 키에 담긴 5개의 점수를 평균하여 얻을 수 있음
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


- cross_validate() 는 훈련 세트를 섞어 폴드를 나누지 않음 

- 앞에서는 train_test_split() 함수로 전체 데이터를 섞은 후 훈련 세트를 준비했기 때문에 섞을 필요가 없었음.

- 만약 교차 검증을 할 때 훈련 세트를 섞으려면 분할기(Splitter)를 지정해야 함.

In [10]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [11]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


- 이제 결정 트리의 매개변수 값을 바꿔가며 가장 좋은 성능이 나오는 모델을 찾아봅시다.

- 이 때, 테스트 세트를 사용하지 않고 교차 검증을 통해서 좋은 모델을 고르면 됩니다.

## 하이퍼파라미터 튜닝